<a href="https://colab.research.google.com/github/LukasEder1/CKE-Evaluation/blob/master/CreateKeywordCollections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/LukasEder1/CKE-Evaluation.git"
%cd "/content/CKE-Evaluation"

Cloning into 'CKE-Evaluation'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 34 (delta 5), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), 170.52 KiB | 2.13 MiB/s, done.
/content/CKE-Evaluation


In [2]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-zsaulavc
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-zsaulavc
  Resolved https://github.com/boudinfl/pke.git to commit 8f1d05dcc52041c9920ba0f9d5231fe6086d12c4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Keyword Collection Creation

In [29]:
from contrastive_keyword_extraction import contrastive_extraction, final_score
import sqlite3
import pandas as pd
from cleantext import clean
from baselines import *
from tqdm import tqdm
import string
import pickle
import sentence_comparision
import sentence_importance
import summary
import utilities
import keyword_extraction
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
with open("policy_data.pkl", "rb") as file:
  policies = pickle.load(file)

In [30]:
def create_collection(sites, 
                      ke_extractor = keyword_extraction.extract_yake, 
                      num_keywords=10,
                      max_ngram=2, 
                      sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                      importance_estimator = sentence_importance.text_rank_importance,
                      use_furthest=False, 
                      name_prefix="",
                      make_data_persistent=False, 
                      path="dataframes",
                      threshold=0.6,
                      stopwords=[],
                      combinator=utilities.alpha_combination,
                      gamma = 0.5,
                      num_splits=1,
                      matching_model="all-MiniLM-L6-v2"):
    
    for i in tqdm(sites.keys()):

        documents = sites[i]
        
        # run CKE-pipeline
        # Extract Keywords, and Matched sentences
        keywords, matched_dict, changed_indices, additions, deletions, new_indices, ranking, removed, matched_indices, unified_delitions = contrastive_extraction(documents, 
                                                                            max_ngram=max_ngram,
                                                                            min_ngram=1, 
                                                                            show_changes=False, 
                                                                            symbols_to_remove=string.punctuation,
                                                                            importance_estimator=importance_estimator,
                                                                            match_sentences=sentence_matcher,
                                                                            threshold=threshold,
                                                                            extra_stopwords=stopwords,
                                                                            top_k=int(num_splits),
                                                                            combinator=combinator,
                                                                            alpha_gamma=gamma,
                                                                            matching_model=matching_model)
        
        

    
        # create itermediate
        kws, scores = extract_from_dict(keywords)
        
        pipeline_frame = pd.DataFrame({'keyword': kws, 'score': scores})
        
        #extractor = lambda x: ke_extractor(x, max_ngram_size=max_ngram, numOfKeywords=num_keywords)
        
        # create CKE on the specified baseline
        baseline_keywords = baseline_diff_content(additions, unified_delitions, ke_extractor, num_keywords, max_ngram)
        
        baseline_kws, baseline_scores = extract_from_tuple_list(baseline_keywords)
        
        baseline_frame1 = pd.DataFrame({'keyword': baseline_kws, 'score': baseline_scores})
        
        
        # create CKE for baseline method 2
        baseline_keywords2 = baseline_keywords_in_diff(documents, ke_extractor, additions, deletions, candidates=50, max_ngram=max_ngram)
        
        baseline_kws2, baseline_scores2 = extract_from_dict(baseline_keywords2)
        
        baseline_frame2 = pd.DataFrame({'keyword': baseline_kws2, 'score': baseline_scores2})
        
        # Baseline 3
        baseline_keywords3 = baseline3(documents, additions, unified_delitions, max_ngram)
        
        baseline_kws3, baseline_scores3 = extract_from_dict(baseline_keywords3)
        
        baseline_frame3 = pd.DataFrame({'keyword': baseline_kws3, 'score': baseline_scores3})

        # Baseline 4
        baseline_keywords4 = baseline4(documents, max_ngram, stopwords)
        
        baseline_kws4, baseline_scores4 = extract_from_dict(baseline_keywords4)
        
        baseline_frame4 = pd.DataFrame({'keyword': baseline_kws4, 'score': baseline_scores4})

        # decide, wether to actually save the data
        if make_data_persistent:
            
            pipeline_frame.to_csv(f"{path}/{name_prefix}_pipeline_keywords_{i}.csv", index=False)
            
            baseline_frame1.to_csv(f"{path}/{name_prefix}_baseline1_keywords_{i}.csv", index=False)
            
            baseline_frame2.to_csv(f"{path}/{name_prefix}_baseline2_keywords_{i}.csv", index=False)
            
            baseline_frame3.to_csv(f"{path}/{name_prefix}_baseline3_keywords_{i}.csv", index=False)
            
            baseline_frame4.to_csv(f"{path}/{name_prefix}_baseline4_keywords_{i}.csv", index=False)
    
    return pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4

In [32]:
create_collection(policies, 
                  ke_extractor = keyword_extraction.extract_yake, 
                  num_keywords=15,
                  max_ngram=2, 
                  sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                  importance_estimator = sentence_importance.text_rank_importance,
                  use_furthest=False, 
                  name_prefix="standard",
                  make_data_persistent=True, 
                  path="Dataframes",
                  threshold=0.65,
                  stopwords=nltk.corpus.stopwords.words("english"),
                  combinator=utilities.alpha_combination,
                  gamma = 0.5,
                  num_splits=1,
                  matching_model ="msmarco-distilbert-base-v4")

  4%|▍         | 1/25 [00:27<10:50, 27.09s/it]


KeyboardInterrupt: ignored

In [ ]:
from itertools import product


def cartesian_product(params):
    
    # gett all possible combinations
    return list(product(*params.values()))



# Parameters to Use

In [ ]:
parameters = {"matcher": [sentence_comparision.match_sentences_semantic_search,
                         sentence_comparision.match_sentences_tfidf_weighted],
              
             "ie": [sentence_importance.text_rank_importance,
                    sentence_importance.yake_weighted_importance],
              
             "threshold": [0.5, 0.6, 0.7]
              
             }


In [ ]:
len(cartesian_product(parameters))

12

In [ ]:
def create_cartesian_collection(params, 
                                sites, 
                                baseline_ke_extractor = keyword_extraction.extract_yake, 
                                num_keywords=10, 
                                max_ngram=2, 
                                use_furthest=False,
                                make_data_persistent=False,
                                file_prefix = "combination",
                                path="dataframes",
                                compare_k = 15):
            
        
    combinations = cartesian_product(params)
    
    number_of_combinations = len(combinations)
    
    count = 0
    
    for combination in combinations:
        
        matcher, threshold = combination
        
        print(f"Contrastive Keyword Extraction pipeline is being ran with combination {count}:")
        
        pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4 = create_collection( sites = sites, 
                                                                  ke_extractor = baseline_ke_extractor, 
                                                                  num_keywords = num_keywords, 
                                                                  max_ngram = max_ngram, 
                                                                  sentence_matcher = matcher,
                                                                  use_furthest = use_furthest,
                                                                  name_prefix=f"{file_prefix}_{count}",
                                                                  make_data_persistent=make_data_persistent,
                                                                  path=path,
                                                                  threshold=threshold)
        
      
        
        count += 1
        

In [ ]:
create_cartesian_collection(parameters,
                            policies
                            baseline_ke_extractor = keyword_extraction.extract_yake,
                            num_keywords=20,
                            max_ngram=2,
                            use_furthest=True, # only compare the first and last document
                            make_data_persistent=True,
                            file_prefix="threshold",
                            path="Combinations")